# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The default looks reasonable, although I have some doubts as to whether it makes sense to discard the data around the edge in this case. 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

import nltk
nltk.download('punkt')

## TODO #1: Select appropriate values for the Python variables below.
batch_size=10
#batch_size = 64          # batch size, we have 10GB of GPU memory, let's use it
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = False    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, max_batch_size=batch_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) + list(encoder.bn.parameters())

# TODO #4: Define the optimizer.
#optimizer = torch.optim.SGD(params, lr=0.01)

# this data is probably pretty sparse, and defaults are probably ok
#http://ruder.io/optimizing-gradient-descent/
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /home/sthenc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.52s)
creating index...


  0%|          | 817/414113 [00:00<00:50, 8165.94it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:42<00:00, 9735.74it/s] 


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [3]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# temporary
%load_ext autoreload
%autoreload 2

from model import EncoderCNN, DecoderRNN

# Open the training log file.
f = open(log_file, 'w')

## running the training locally
#old_time = time.time()
#response = requests.request("GET", 
#                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                            headers={"Metadata-Flavor":"Google"})


for epoch in range(1, num_epochs+1):

    for i_step in range(1, total_step+1):
        
        ## running the training locally
        #if time.time() - old_time > 60:
        #    old_time = time.time()
        #    requests.request("POST", 
        #                     "https://nebula.udacity.com/api/v1/remote/keep-alive", 
        #                     headers={'Authorization': "STAR " + response.text})

        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)

        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()

        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)

        # Calculate the batch loss.
        loss = criterion(outputs.contiguous().view(-1, vocab_size), captions.contiguous().view(-1))

        # Backward pass.
        loss.backward(retain_graph=True)

        # Update the parameters in the optimizer.
        optimizer.step()

        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))

        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()

        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()

        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)

    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/41412], Loss: 4.2696, Perplexity: 71.4945
Epoch [1/3], Step [200/41412], Loss: 4.0440, Perplexity: 57.05449
Epoch [1/3], Step [300/41412], Loss: 3.3515, Perplexity: 28.54571
Epoch [1/3], Step [400/41412], Loss: 2.6613, Perplexity: 14.31442
Epoch [1/3], Step [500/41412], Loss: 3.8998, Perplexity: 49.3939
Epoch [1/3], Step [600/41412], Loss: 3.2152, Perplexity: 24.90838
Epoch [1/3], Step [700/41412], Loss: 3.5679, Perplexity: 35.4411
Epoch [1/3], Step [800/41412], Loss: 3.5968, Perplexity: 36.48171
Epoch [1/3], Step [900/41412], Loss: 3.0525, Perplexity: 21.1674
Epoch [1/3], Step [1000/41412], Loss: 2.9991, Perplexity: 20.0665
Epoch [1/3], Step [1100/41412], Loss: 3.1469, Perplexity: 23.26359
Epoch [1/3], Step [1200/41412], Loss: 3.8761, Perplexity: 48.2379
Epoch [1/3], Step [1300/41412], Loss: 3.1357, Perplexity: 23.0048
Epoch [1/3], Step [1400/41412], Loss: 3.5219, Perplexity: 33.84868
Epoch [1/3], Step [1500/41412], Loss: 3.1983, Perplexity: 24.4911
Epoch [1/3],

Epoch [1/3], Step [12400/41412], Loss: 2.3371, Perplexity: 10.3516
Epoch [1/3], Step [12500/41412], Loss: 2.5113, Perplexity: 12.3205
Epoch [1/3], Step [12600/41412], Loss: 2.6599, Perplexity: 14.2945
Epoch [1/3], Step [12700/41412], Loss: 1.9965, Perplexity: 7.36350
Epoch [1/3], Step [12800/41412], Loss: 2.5177, Perplexity: 12.4006
Epoch [1/3], Step [12900/41412], Loss: 2.4616, Perplexity: 11.7234
Epoch [1/3], Step [13000/41412], Loss: 2.1033, Perplexity: 8.19294
Epoch [1/3], Step [13100/41412], Loss: 2.8980, Perplexity: 18.1384
Epoch [1/3], Step [13200/41412], Loss: 2.2467, Perplexity: 9.45692
Epoch [1/3], Step [13300/41412], Loss: 2.1315, Perplexity: 8.42710
Epoch [1/3], Step [13400/41412], Loss: 1.9843, Perplexity: 7.27402
Epoch [1/3], Step [13500/41412], Loss: 2.3446, Perplexity: 10.4288
Epoch [1/3], Step [13600/41412], Loss: 2.2258, Perplexity: 9.26064
Epoch [1/3], Step [13700/41412], Loss: 2.4618, Perplexity: 11.7257
Epoch [1/3], Step [13800/41412], Loss: 2.5169, Perplexity: 12.

Epoch [1/3], Step [24600/41412], Loss: 2.5629, Perplexity: 12.9736
Epoch [1/3], Step [24700/41412], Loss: 2.7914, Perplexity: 16.3032
Epoch [1/3], Step [24800/41412], Loss: 1.7684, Perplexity: 5.86133
Epoch [1/3], Step [24900/41412], Loss: 2.1037, Perplexity: 8.19633
Epoch [1/3], Step [25000/41412], Loss: 2.2479, Perplexity: 9.46774
Epoch [1/3], Step [25100/41412], Loss: 2.7326, Perplexity: 15.3723
Epoch [1/3], Step [25200/41412], Loss: 2.3651, Perplexity: 10.6450
Epoch [1/3], Step [25300/41412], Loss: 2.3281, Perplexity: 10.2589
Epoch [1/3], Step [25400/41412], Loss: 2.4536, Perplexity: 11.6301
Epoch [1/3], Step [25500/41412], Loss: 2.4304, Perplexity: 11.3638
Epoch [1/3], Step [25600/41412], Loss: 2.5535, Perplexity: 12.8518
Epoch [1/3], Step [25700/41412], Loss: 1.6592, Perplexity: 5.25535
Epoch [1/3], Step [25800/41412], Loss: 2.4214, Perplexity: 11.2614
Epoch [1/3], Step [25900/41412], Loss: 2.0638, Perplexity: 7.87605
Epoch [1/3], Step [26000/41412], Loss: 1.8483, Perplexity: 6.3

Epoch [1/3], Step [36800/41412], Loss: 1.9753, Perplexity: 7.20906
Epoch [1/3], Step [36900/41412], Loss: 2.7967, Perplexity: 16.3899
Epoch [1/3], Step [37000/41412], Loss: 2.4969, Perplexity: 12.1443
Epoch [1/3], Step [37100/41412], Loss: 2.7478, Perplexity: 15.6086
Epoch [1/3], Step [37200/41412], Loss: 2.7636, Perplexity: 15.8574
Epoch [1/3], Step [37300/41412], Loss: 1.8378, Perplexity: 6.28276
Epoch [1/3], Step [37400/41412], Loss: 2.3406, Perplexity: 10.3876
Epoch [1/3], Step [37500/41412], Loss: 2.3271, Perplexity: 10.2485
Epoch [1/3], Step [37600/41412], Loss: 2.3625, Perplexity: 10.6178
Epoch [1/3], Step [37700/41412], Loss: 2.1395, Perplexity: 8.49484
Epoch [1/3], Step [37800/41412], Loss: 2.1816, Perplexity: 8.86036
Epoch [1/3], Step [37900/41412], Loss: 1.7540, Perplexity: 5.77792
Epoch [1/3], Step [38000/41412], Loss: 2.5743, Perplexity: 13.1222
Epoch [1/3], Step [38100/41412], Loss: 2.2110, Perplexity: 9.12503
Epoch [1/3], Step [38200/41412], Loss: 2.5969, Perplexity: 13.

Epoch [2/3], Step [7700/41412], Loss: 2.1014, Perplexity: 8.17791
Epoch [2/3], Step [7800/41412], Loss: 2.2806, Perplexity: 9.78246
Epoch [2/3], Step [7900/41412], Loss: 2.5248, Perplexity: 12.4879
Epoch [2/3], Step [8000/41412], Loss: 1.7994, Perplexity: 6.04626
Epoch [2/3], Step [8100/41412], Loss: 2.1797, Perplexity: 8.84339
Epoch [2/3], Step [8200/41412], Loss: 2.4439, Perplexity: 11.5174
Epoch [2/3], Step [8300/41412], Loss: 2.3839, Perplexity: 10.8470
Epoch [2/3], Step [8400/41412], Loss: 1.6283, Perplexity: 5.09541
Epoch [2/3], Step [8500/41412], Loss: 1.9721, Perplexity: 7.18608
Epoch [2/3], Step [8600/41412], Loss: 2.1364, Perplexity: 8.46873
Epoch [2/3], Step [8700/41412], Loss: 2.1108, Perplexity: 8.25522
Epoch [2/3], Step [8800/41412], Loss: 1.8769, Perplexity: 6.53331
Epoch [2/3], Step [8900/41412], Loss: 1.8846, Perplexity: 6.58371
Epoch [2/3], Step [9000/41412], Loss: 2.1827, Perplexity: 8.87052
Epoch [2/3], Step [9100/41412], Loss: 2.4592, Perplexity: 11.6959
Epoch [2/3

...

Epoch [2/3], Step [32100/41412], Loss: 2.1392, Perplexity: 8.49290
Epoch [2/3], Step [32200/41412], Loss: 2.3417, Perplexity: 10.3984
Epoch [2/3], Step [32300/41412], Loss: 1.7130, Perplexity: 5.54554
Epoch [2/3], Step [32400/41412], Loss: 2.1831, Perplexity: 8.87413
Epoch [2/3], Step [32500/41412], Loss: 2.3657, Perplexity: 10.6514
Epoch [2/3], Step [32600/41412], Loss: 1.9720, Perplexity: 7.18490
Epoch [2/3], Step [32700/41412], Loss: 1.6328, Perplexity: 5.11823
Epoch [2/3], Step [32800/41412], Loss: 1.8592, Perplexity: 6.41887
Epoch [2/3], Step [32900/41412], Loss: 2.1007, Perplexity: 8.17223
Epoch [2/3], Step [33000/41412], Loss: 2.4924, Perplexity: 12.0906
Epoch [2/3], Step [33100/41412], Loss: 1.8690, Perplexity: 6.48151
Epoch [2/3], Step [33200/41412], Loss: 1.9130, Perplexity: 6.77310
Epoch [2/3], Step [33300/41412], Loss: 2.5205, Perplexity: 12.4345
Epoch [2/3], Step [33400/41412], Loss: 1.5852, Perplexity: 4.88020
Epoch [2/3], Step [33500/41412], Loss: 2.0922, Perplexity: 8.1

Epoch [3/3], Step [2900/41412], Loss: 2.0211, Perplexity: 7.54666
Epoch [3/3], Step [3000/41412], Loss: 2.1555, Perplexity: 8.63239
Epoch [3/3], Step [3100/41412], Loss: 2.1748, Perplexity: 8.80026
Epoch [3/3], Step [3200/41412], Loss: 1.9004, Perplexity: 6.68841
Epoch [3/3], Step [3300/41412], Loss: 2.5239, Perplexity: 12.4769
Epoch [3/3], Step [3400/41412], Loss: 1.7351, Perplexity: 5.66925
Epoch [3/3], Step [3500/41412], Loss: 2.2424, Perplexity: 9.41573
Epoch [3/3], Step [3600/41412], Loss: 2.6455, Perplexity: 14.0904
Epoch [3/3], Step [3700/41412], Loss: 2.1382, Perplexity: 8.48417
Epoch [3/3], Step [3800/41412], Loss: 2.2261, Perplexity: 9.26401
Epoch [3/3], Step [3900/41412], Loss: 1.9332, Perplexity: 6.91138
Epoch [3/3], Step [4000/41412], Loss: 1.7474, Perplexity: 5.73992
Epoch [3/3], Step [4100/41412], Loss: 2.3000, Perplexity: 9.97446
Epoch [3/3], Step [4200/41412], Loss: 2.2915, Perplexity: 9.88949
Epoch [3/3], Step [4300/41412], Loss: 1.9118, Perplexity: 6.76525
Epoch [3/3

Epoch [3/3], Step [15200/41412], Loss: 2.3313, Perplexity: 10.2911
Epoch [3/3], Step [15300/41412], Loss: 2.2315, Perplexity: 9.31354
Epoch [3/3], Step [15400/41412], Loss: 1.4803, Perplexity: 4.39412
Epoch [3/3], Step [15500/41412], Loss: 2.1931, Perplexity: 8.96253
Epoch [3/3], Step [15600/41412], Loss: 2.5301, Perplexity: 12.5541
Epoch [3/3], Step [15700/41412], Loss: 2.0332, Perplexity: 7.63883
Epoch [3/3], Step [15800/41412], Loss: 2.2818, Perplexity: 9.79418
Epoch [3/3], Step [15900/41412], Loss: 1.8545, Perplexity: 6.38853
Epoch [3/3], Step [16000/41412], Loss: 1.5414, Perplexity: 4.67136
Epoch [3/3], Step [16100/41412], Loss: 2.0676, Perplexity: 7.90589
Epoch [3/3], Step [16200/41412], Loss: 2.0283, Perplexity: 7.60118
Epoch [3/3], Step [16300/41412], Loss: 1.8137, Perplexity: 6.13299
Epoch [3/3], Step [16400/41412], Loss: 2.2516, Perplexity: 9.50300
Epoch [3/3], Step [16500/41412], Loss: 1.9125, Perplexity: 6.77036
Epoch [3/3], Step [16600/41412], Loss: 2.0162, Perplexity: 7.5

Epoch [3/3], Step [27400/41412], Loss: 2.1061, Perplexity: 8.21588
Epoch [3/3], Step [27500/41412], Loss: 2.5147, Perplexity: 12.36262
Epoch [3/3], Step [27600/41412], Loss: 2.2110, Perplexity: 9.12526
Epoch [3/3], Step [27700/41412], Loss: 1.9271, Perplexity: 6.86953
Epoch [3/3], Step [27800/41412], Loss: 1.7101, Perplexity: 5.52975
Epoch [3/3], Step [27900/41412], Loss: 2.1111, Perplexity: 8.25731
Epoch [3/3], Step [28000/41412], Loss: 2.0511, Perplexity: 7.77672
Epoch [3/3], Step [28100/41412], Loss: 2.4810, Perplexity: 11.9531
Epoch [3/3], Step [28200/41412], Loss: 1.7948, Perplexity: 6.01800
Epoch [3/3], Step [28300/41412], Loss: 1.9310, Perplexity: 6.89616
Epoch [3/3], Step [28400/41412], Loss: 2.6673, Perplexity: 14.4013
Epoch [3/3], Step [28500/41412], Loss: 1.7339, Perplexity: 5.66287
Epoch [3/3], Step [28600/41412], Loss: 1.7867, Perplexity: 5.97004
Epoch [3/3], Step [28700/41412], Loss: 1.8895, Perplexity: 6.61588
Epoch [3/3], Step [28800/41412], Loss: 1.6389, Perplexity: 5.

Epoch [3/3], Step [39600/41412], Loss: 2.0859, Perplexity: 8.05151
Epoch [3/3], Step [39700/41412], Loss: 2.4824, Perplexity: 11.9695
Epoch [3/3], Step [39800/41412], Loss: 2.1294, Perplexity: 8.40952
Epoch [3/3], Step [39900/41412], Loss: 2.5433, Perplexity: 12.7221
Epoch [3/3], Step [40000/41412], Loss: 2.2637, Perplexity: 9.61874
Epoch [3/3], Step [40100/41412], Loss: 2.1867, Perplexity: 8.90568
Epoch [3/3], Step [40200/41412], Loss: 2.0229, Perplexity: 7.56040
Epoch [3/3], Step [40300/41412], Loss: 1.9445, Perplexity: 6.99012
Epoch [3/3], Step [40400/41412], Loss: 2.4210, Perplexity: 11.2576
Epoch [3/3], Step [40500/41412], Loss: 2.1203, Perplexity: 8.33396
Epoch [3/3], Step [40600/41412], Loss: 1.8267, Perplexity: 6.21358
Epoch [3/3], Step [40700/41412], Loss: 2.6225, Perplexity: 13.7695
Epoch [3/3], Step [40800/41412], Loss: 2.1856, Perplexity: 8.89560
Epoch [3/3], Step [40900/41412], Loss: 1.7097, Perplexity: 5.52741
Epoch [3/3], Step [41000/41412], Loss: 1.9982, Perplexity: 7.3

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.